## AlphaTims

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import panel as pn

pn.extension('bokeh')

In [ ]:
css = '''
.bk.opt {
    position: relative;
    display: block;
    left: 75px;
    top: 0px;
    width: 80px;
    height: 80px;
}

h1 {
    color: #045082;
    font-size: 45px;
    line-height: 0.6;
    text-align: center;
}

.bk.main-part {
    background-color: #EAEAEA;
    font-size: 17px;
    line-height: 23px;
    letter-spacing: 0px;
    font-weight: 500;
    color: #045082;
    text-align: center;
    position: relative !important;
    margin-left: auto;
    margin-right: auto;
    width: 40%;
}

.bk-root .bk-btn-primary {
    background-color: #045082;
    font-size: 11px;
    font-weight: 700;
    text-transform: uppercase;
    letter-spacing: 1.5px;
}

'''
pn.extension(raw_css=[css])

In [ ]:
### HEADER

header_titel = pn.pane.Markdown(
    '# AlphaTims', 
    width=1250
)
mpi_biochem_logo = pn.pane.PNG(
    'img/mpi_logo.png', 
    link_url='https://www.biochem.mpg.de/en', 
    width=60, 
    height=60,
    align='start'
)
mpi_logo = pn.pane.JPG(
    'img/max-planck-gesellschaft.jpg', 
    link_url='https://www.biochem.mpg.de/en',
    height=62,
    embed=True,
    width=62,
    margin=(5, 0, 0, 5),
    css_classes=['opt']
)
github_logo = pn.pane.PNG(
    'img/github.png',
    link_url='https://github.com/MannLabs/alphatims',
    height=70,
)

header = pn.Row(
    mpi_biochem_logo,
    mpi_logo,
    header_titel,
    github_logo,
    height=73
)

In [ ]:
### MAIN PART

project_description = pn.pane.Markdown(
    """### AlphaTIMS an open-source Python package for fast accessing Bruker TimsTOF data. It provides a very efficient indexed data structure that allows to access four-dimensional TIMS-time of flight data in the standard numerical python (NumPy) manner. AlphaTIMS is a key enabling tool to deal with the large and high dimensional TIMS data.""",
    margin=(10, 0, 0, 0), 
    css_classes=['main-part'], 
    width=615
)

divider_descr = pn.pane.HTML(
    '<hr style="height: 6px; border:none; background-color: #045082; width: 610px">', 
    width=1510, 
    align='center'
)

upload_file = pn.widgets.TextInput(
    name='Specify an experimental file:',
    placeholder='Enter the whole path to Bruker .d folder',
    width=400,
    margin=15
)

upload_button = pn.widgets.Button(
    name='Upload  Data', 
    button_type='primary', 
    height=35,
    width=100,
    margin=(32,20,0,20)
)

upload_spinner = pn.indicators.LoadingSpinner(
    value=False,
    bgcolor='light',
    color='secondary',
    margin=(30,20,0,10),
    width=40, 
    height=40
)

main_part = pn.Column(
    project_description,
    divider_descr,
    pn.Row(
        upload_file,
        upload_button,
        upload_spinner,
        align='center'
    ),
    background='#eaeaea',
    width=1510,
    margin=(5, 0, 10, 0)
)

In [ ]:
layout = pn.Column(
    header,
    main_part
)
layout.servable()